In [3]:
%load_ext autoreload
%autoreload 2
from keras.models import Sequential
from keras.layers import Dense, Dropout
from scikeras.wrappers import KerasClassifier
from tensorflow.keras import regularizers
import tensorflow as tf
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer, MissingIndicator, KNNImputer
import numpy as np
import pandas as pd
from preprocessing import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
df_features, df_target = get_datasets()

viento_trigonometrico(df_features)
df_features.drop(
    columns=[
        "dia",
        "cos_rafaga_viento_max_direccion",
        "sin_rafaga_viento_max_direccion",
        "barrio",
        "velocidad_viento_temprano",
        "velocidad_viento_tarde",
        "cos_viento_tarde"
    ],
        inplace=True)


X_train, X_test, Y_train, Y_test = train_test_split(df_features, df_target, stratify=df_target.values, test_size=0.35, random_state=0)

imputer = IterativeImputer(max_iter=10000)
imputer.fit(X_train)
X_train = imputer.transform(X_train)
X_test = imputer.transform(X_test)

Y_train = np.ravel(Y_train)
Y_test = np.ravel(Y_test)

In [8]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [9]:
model = Sequential()
model.add(Dense(24, input_dim=X_train.shape[1], activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(8, activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(4, activation="tanh"))
model.add(Dense(1, activation='sigmoid'))

In [23]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy", tf.keras.metrics.AUC()])

In [27]:
parameters = {"epochs": np.arange(100, 500, 100), "batch_size": np.arange(500,5000,100)}
grid = GridSearchCV(KerasClassifier(model), param_grid=parameters, scoring="roc_auc")

In [ ]:
grid.fit(X_train, Y_train)

In [ ]:
history = model.fit(X_train, Y_train, epochs=25, batch_size=50, verbose=1)

In [17]:
model.evaluate(X_test, Y_test)

1120/1120 [==============================] - 1s 581us/step - loss: 0.3398 - accuracy: 0.8530 - auc_3: 0.8805


[0.33982914686203003, 0.8530389070510864, 0.8804783821105957]